In [ ]:
import sys
sys.path.append("../src")

import requests

from tonpy import LiteClient, Cell, get_block_info, BlockId, BlockIdExt, \
        Address, Emulator, begin_cell, StackEntry, VmDict, CellSlice

from tonpy.utils.shard_account import shard_is_ancestor, shard_child
from tonpy.libs.python_ton import Bits256
from tonpy.autogen.block import Account, Transaction, Block, BlockInfo, BlockExtra, MessageAny, AccountState
from tqdm import tqdm
from tonpy.utils.chunks import chunks
from multiprocess.pool import ThreadPool

In [ ]:
lc = LiteClient(mode='roundrobin')

In [ ]:
mc_blocks = []

In [ ]:
start = lc.get_masterchain_info_ext()

In [ ]:
mc_hashes = []

In [ ]:
def f(c):
    lc = LiteClient(mode='roundrobin')
    answer = []
    for i in tqdm(c):
        answer.append(lc.lookup_block(BlockId(-1, 0x8000000000000000, i)))
    return answer

In [ ]:
mc_seqnos = list(chunks(list(range(start.last.id.seqno - 150000, start.last.id.seqno)), 100))

print("Total: ", len(mc_seqnos))
with ThreadPool(5) as pool:
    results = pool.imap_unordered(f, mc_seqnos)

    for result in tqdm(results, desc="[multithread] Load blocks",
                       total=len(mc_seqnos)):
        mc_hashes.extend(result)